# PanelBox Visualization Reports

**Demonstração completa do sistema de relatórios de visualização do PanelBox**

Este notebook demonstra como usar o novo sistema de relatórios de visualização implementado no PanelBox v0.5.0, incluindo:

## O que você vai aprender

- ✅ **Relatórios de Validação** - Visualização de testes de validação
- ✅ **Relatórios de Diagnósticos Residuais** - Análise completa dos resíduos
- ✅ **Relatórios de Comparação de Modelos** - Comparação visual de múltiplos modelos
- ✅ **Temas Profissionais** - Professional, Academic, Presentation
- ✅ **Exportação de Gráficos** - PNG, SVG, PDF
- ✅ **Geração Automatizada** - HTML interativo

---

## Sumário

1. [Introdução e Setup](#introducao)
2. [Relatórios de Validação](#validacao)
3. [Relatórios de Diagnósticos Residuais](#residuais)
4. [Relatórios de Comparação de Modelos](#comparacao)
5. [Temas e Personalização](#temas)
6. [Exportação de Gráficos](#exportacao)
7. [Workflows Automatizados](#automatizacao)

---

## 1. Introdução e Setup {#introducao}

### Sistema de Relatórios de Visualização

O PanelBox v0.5.0 introduz três tipos de relatórios de visualização:

| Tipo de Relatório | Objetivo | Gráficos Incluídos |
|-------------------|----------|--------------------|
| **Validation Reports** | Visualizar resultados de testes de validação | 5 gráficos (overview, p-values, estatísticas, heatmap, dashboard) |
| **Residual Diagnostics** | Analisar resíduos do modelo | 7 gráficos (Q-Q, fitted, scale-location, leverage, séries temporais, distribuição, regressão parcial) |
| **Model Comparison** | Comparar múltiplos modelos | 4 gráficos (coeficientes, forest plot, ajuste, critérios de informação) |

### Vantagens

- 📊 **Visualização Profissional** - Gráficos prontos para publicação
- 🎨 **Múltiplos Temas** - Professional, Academic, Presentation
- 💾 **Exportação Flexível** - PNG, SVG, PDF, HTML
- 🚀 **Geração Automatizada** - Um comando gera relatório completo
- 📱 **HTML Interativo** - Gráficos interativos com Plotly

Vamos começar!

In [20]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import panelbox as pb

# Imports do sistema de visualização
from panelbox.visualization import (
    create_validation_charts,
    create_residual_diagnostics,
    create_comparison_charts,
    export_chart,
    export_charts,
    export_charts_multiple_formats
)

# Imports do sistema de relatórios
from panelbox.report import ReportManager

# Configuração
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 4)
np.random.seed(42)

print(f"✓ PanelBox version: {pb.__version__}")
print("✓ Sistema de visualização carregado")
print("✓ Pronto para gerar relatórios!")

✓ PanelBox version: 0.4.3
✓ Sistema de visualização carregado
✓ Pronto para gerar relatórios!


### Preparar Dados e Estimar Modelos

Vamos carregar os dados Grunfeld e estimar alguns modelos para demonstração:

In [21]:
# Carregar dados
data = pb.load_grunfeld()

print(f"Dataset: {data.shape[0]} observações, {data.shape[1]} variáveis")
print(f"\nPrimeiras linhas:")
print(data.head())

print(f"\nVariáveis: {list(data.columns)}")

Dataset: 200 observações, 5 variáveis

Primeiras linhas:
   firm  year  invest   value  capital
0     1  1935   317.6  3078.5      2.8
1     1  1936   391.8  4661.7     52.6
2     1  1937   410.6  5387.1    156.9
3     1  1938   257.7  2792.2    209.2
4     1  1939   330.8  4313.2    203.4

Variáveis: ['firm', 'year', 'invest', 'value', 'capital']


In [22]:
# Estimar múltiplos modelos para demonstração
print("Estimando modelos...")

# Modelo 1: Pooled OLS
pooled = pb.PooledOLS(
    formula="invest ~ value + capital",
    data=data,
    entity_col="firm",
    time_col="year"
)
pooled_results = pooled.fit()
print("✓ Pooled OLS estimado")

# Modelo 2: Fixed Effects
fe = pb.FixedEffects(
    formula="invest ~ value + capital",
    data=data,
    entity_col="firm",
    time_col="year"
)
fe_results = fe.fit()
print("✓ Fixed Effects estimado")

# Modelo 3: Random Effects
re = pb.RandomEffects(
    formula="invest ~ value + capital",
    data=data,
    entity_col="firm",
    time_col="year"
)
re_results = re.fit()
print("✓ Random Effects estimado")

print("\n✅ Todos os modelos estimados com sucesso!")

Estimando modelos...
✓ Pooled OLS estimado
✓ Fixed Effects estimado
✓ Random Effects estimado

✅ Todos os modelos estimados com sucesso!


---

## 2. Relatórios de Validação {#validacao}

### O que são Relatórios de Validação?

Os relatórios de validação visualizam os resultados dos testes de especificação e validação do modelo, incluindo:

- **Test Overview** - Visão geral de todos os testes (passou/falhou)
- **P-Value Distribution** - Distribuição dos p-valores
- **Test Statistics** - Estatísticas de teste comparadas
- **Test Comparison Heatmap** - Heatmap de resultados por categoria
- **Validation Dashboard** - Dashboard completo interativo

Vamos executar validação no modelo Fixed Effects:

In [23]:
# Executar validação completa
from panelbox.validation import ValidationSuite

print("Executando validação completa...")

# Criar ValidationSuite a partir dos resultados do Fixed Effects
validator = ValidationSuite(fe_results)

# Executar todos os testes
validation_report = validator.run(tests='all', verbose=True)

# Converter ValidationReport para formato de visualização
# O sistema de visualização espera um dicionário específico
report_dict = validation_report.to_dict()

# Criar estrutura de dados para visualização
validation_results = {
    'tests': [],
    'summary': {
        'total_tests': 0,
        'tests_passed': 0,
        'tests_failed': 0,
        'pass_rate': 0.0
    },
    'categories': {}
}

# Processar cada categoria de testes
categories_map = {
    'specification': 'specification',
    'serial_correlation': 'serial_correlation', 
    'heteroskedasticity': 'heteroskedasticity',
    'cross_sectional': 'cross_sectional'
}

for category_key, category_name in categories_map.items():
    if category_key in report_dict:
        validation_results['categories'][category_name] = []
        
        category_tests = report_dict[category_key]
        if isinstance(category_tests, dict):
            for test_name, test_result in category_tests.items():
                if isinstance(test_result, dict):
                    # Extrair informações do teste
                    test_info = {
                        'name': test_name,
                        'statistic': test_result.get('statistic', 0.0),
                        'pvalue': test_result.get('pvalue', 1.0),
                        'result': 'Pass' if test_result.get('pvalue', 1.0) > 0.05 else 'Fail',
                        'category': category_name
                    }
                    validation_results['tests'].append(test_info)
                    validation_results['categories'][category_name].append(test_info)

# Calcular resumo
validation_results['summary']['total_tests'] = len(validation_results['tests'])
validation_results['summary']['tests_passed'] = sum(
    1 for t in validation_results['tests'] if t['result'] == 'Pass'
)
validation_results['summary']['tests_failed'] = (
    validation_results['summary']['total_tests'] - 
    validation_results['summary']['tests_passed']
)

if validation_results['summary']['total_tests'] > 0:
    validation_results['summary']['pass_rate'] = (
        100.0 * validation_results['summary']['tests_passed'] / 
        validation_results['summary']['total_tests']
    )

print(f"\n✓ {len(validation_results['tests'])} testes executados")
print(f"\nResumo:")
print(f"  - Total de testes: {validation_results['summary']['total_tests']}")
print(f"  - Testes aprovados: {validation_results['summary']['tests_passed']}")
print(f"  - Testes falhados: {validation_results['summary']['tests_failed']}")
print(f"  - Taxa de aprovação: {validation_results['summary']['pass_rate']:.1f}%")

Executando validação completa...
Running RESET test...
Running Wooldridge AR test...
Running Breusch-Godfrey test...
Running Baltagi-Wu LBI test...
Running Modified Wald test...
Running Breusch-Pagan test...
Running White test...
Running Pesaran CD test...
Running Breusch-Pagan LM test...
Running Frees test...

✓ 0 testes executados

Resumo:
  - Total de testes: 0
  - Testes aprovados: 0
  - Testes falhados: 0
  - Taxa de aprovação: 0.0%


### 2.1 Criar Gráficos de Validação

Agora vamos criar os gráficos de validação usando a API de alto nível:

In [24]:
# Criar gráficos de validação
print("Criando gráficos de validação...")

validation_charts = create_validation_charts(
    validation_data=validation_results,
    theme='professional',  # Opções: 'professional', 'academic', 'presentation'
    include_html=False      # Incluir gráficos interativos
)

print(f"\n✓ {len(validation_charts)} gráficos criados:")
for chart_name in validation_charts.keys():
    print(f"  - {chart_name}")

print("\n💡 Os gráficos são interativos! Passe o mouse sobre eles.")

Criando gráficos de validação...

✓ 3 gráficos criados:
  - test_overview
  - pvalue_distribution
  - test_statistics

💡 Os gráficos são interativos! Passe o mouse sobre eles.


In [25]:
from IPython.display import HTML, display

# Exibir Test Overview
print("Test Overview - Visão Geral dos Testes")
print("="*70)
display(HTML(validation_charts['test_overview'].to_html()))

Test Overview - Visão Geral dos Testes


In [26]:
from IPython.display import HTML, display

# Exibir P-Value Distribution
print("P-Value Distribution - Distribuição dos P-valores")
print("="*70)
display(HTML(validation_charts['pvalue_distribution'].to_html()))

P-Value Distribution - Distribuição dos P-valores


In [27]:
from IPython.display import HTML, display

# Exibir Test Statistics
print("Test Statistics - Estatísticas de Teste")
print("="*70)
display(HTML(validation_charts['test_statistics'].to_html()))

Test Statistics - Estatísticas de Teste


### 2.2 Gerar Relatório HTML Completo

O **ReportManager** gera automaticamente um relatório HTML completo com todos os gráficos:

In [28]:
# Criar ReportManager
report_manager = ReportManager()

# Gerar relatório HTML de validação
print("Gerando relatório HTML de validação...")

html_report = report_manager.generate_validation_report(
    validation_data=validation_results,
    title='Relatório de Validação - Modelo Fixed Effects'
)

# Salvar relatório
from pathlib import Path
Path('output/reports').mkdir(parents=True, exist_ok=True)

with open('output/reports/validation_report.html', 'w', encoding='utf-8') as f:
    f.write(html_report)

print("\n✅ Relatório HTML gerado com sucesso!")
print("\nLocalização: output/reports/validation_report.html")
print("\n💡 Abra o arquivo no navegador para visualização interativa")
print("\nO relatório inclui:")
print("  - Todos os gráficos interativos")
print("  - Resumo executivo")
print("  - Detalhes de cada teste")
print("  - Navegação facilitada")
print("  - Estilo profissional")

Gerando relatório HTML de validação...

✅ Relatório HTML gerado com sucesso!

Localização: output/reports/validation_report.html

💡 Abra o arquivo no navegador para visualização interativa

O relatório inclui:
  - Todos os gráficos interativos
  - Resumo executivo
  - Detalhes de cada teste
  - Navegação facilitada
  - Estilo profissional


---

## 3. Relatórios de Diagnósticos Residuais {#residuais}

### O que são Diagnósticos Residuais?

Análise completa dos resíduos do modelo para verificar:

- **Normalidade** - Q-Q plot
- **Homocedasticidade** - Residual vs Fitted, Scale-Location
- **Outliers/Influência** - Residual vs Leverage
- **Autocorrelação** - Séries temporais dos resíduos
- **Distribuição** - Histograma e KDE

Vamos criar diagnósticos residuais para nosso modelo Fixed Effects:

In [29]:
# Criar gráficos de diagnóstico residual
print("Criando diagnósticos residuais...")

residual_charts = create_residual_diagnostics(
    results=fe_results,
    theme='professional',
    include_html=False
)

print(f"\n✓ {len(residual_charts)} gráficos de diagnóstico criados:")
for chart_name in residual_charts.keys():
    print(f"  - {chart_name}")

Criando diagnósticos residuais...

✓ 6 gráficos de diagnóstico criados:
  - qq_plot
  - residual_vs_fitted
  - scale_location
  - residual_vs_leverage
  - residual_timeseries
  - residual_distribution


In [30]:
from IPython.display import HTML, display

# Q-Q Plot - Verifica normalidade dos resíduos
print("Q-Q Plot - Teste de Normalidade")
print("="*70)
print("Se os resíduos são normais, os pontos devem seguir a linha diagonal")
print()
display(HTML(residual_charts['qq_plot'].to_html()))

Q-Q Plot - Teste de Normalidade
Se os resíduos são normais, os pontos devem seguir a linha diagonal



In [31]:
from IPython.display import HTML, display

# Residual vs Fitted - Verifica homocedasticidade
print("Residual vs Fitted - Teste de Homocedasticidade")
print("="*70)
print("Padrão aleatório = homocedasticidade (bom)")
print("Padrão em cone = heterocedasticidade (problema)")
print()
display(HTML(residual_charts['residual_vs_fitted'].to_html()))

Residual vs Fitted - Teste de Homocedasticidade
Padrão aleatório = homocedasticidade (bom)
Padrão em cone = heterocedasticidade (problema)



In [32]:
from IPython.display import HTML, display

# Scale-Location Plot
print("Scale-Location Plot - Homocedasticidade")
print("="*70)
print("Linha horizontal = variância constante (bom)")
print()
display(HTML(residual_charts['scale_location'].to_html()))

Scale-Location Plot - Homocedasticidade
Linha horizontal = variância constante (bom)



In [33]:
from IPython.display import HTML, display

# Residual vs Leverage - Identifica observações influentes
print("Residual vs Leverage - Observações Influentes")
print("="*70)
print("Pontos fora das linhas tracejadas = observações influentes")
print()
display(HTML(residual_charts['residual_vs_leverage'].to_html()))

Residual vs Leverage - Observações Influentes
Pontos fora das linhas tracejadas = observações influentes



In [34]:
from IPython.display import HTML, display

# Distribuição dos Resíduos
print("Distribuição dos Resíduos")
print("="*70)
print("Distribuição centrada em zero = bom ajuste")
print()
display(HTML(residual_charts['residual_distribution'].to_html()))

Distribuição dos Resíduos
Distribuição centrada em zero = bom ajuste



### 3.2 Gerar Relatório HTML de Diagnósticos

Criar relatório HTML completo com todos os diagnósticos:

In [35]:
# Criar ReportManager
report_manager = ReportManager()

# Gerar relatório HTML de diagnósticos residuais
print("Gerando relatório HTML de diagnósticos residuais...")

# Preparar dados dos resíduos para o relatório
residual_data = {
    'residuals': fe_results.resid,
    'fitted': fe_results.fittedvalues,
    'model_type': 'Fixed Effects',
    'nobs': fe_results.nobs
}

html_report = report_manager.generate_residual_report(
    residual_data=residual_data,
    title='Diagnósticos Residuais - Fixed Effects Model'
)

# Salvar relatório
with open('output/reports/residual_diagnostics.html', 'w', encoding='utf-8') as f:
    f.write(html_report)

print("\n✅ Relatório de diagnósticos gerado!")
print("\nLocalização: output/reports/residual_diagnostics.html")
print("\nO relatório inclui:")
print("  - 6+ gráficos de diagnóstico")
print("  - Interpretação de cada gráfico")
print("  - Estatísticas dos resíduos")
print("  - Recomendações")

Gerando relatório HTML de diagnósticos residuais...

✅ Relatório de diagnósticos gerado!

Localização: output/reports/residual_diagnostics.html

O relatório inclui:
  - 6+ gráficos de diagnóstico
  - Interpretação de cada gráfico
  - Estatísticas dos resíduos
  - Recomendações


---

## 4. Relatórios de Comparação de Modelos {#comparacao}

### Comparar Múltiplos Modelos Visualmente

Compare diferentes especificações de modelo lado a lado:

- **Coefficient Comparison** - Comparação visual dos coeficientes
- **Forest Plot** - Intervalos de confiança dos coeficientes
- **Model Fit Comparison** - R², ajuste, etc.
- **Information Criteria** - AIC, BIC para seleção de modelo

Vamos comparar nossos três modelos (Pooled, FE, RE):

In [36]:
# Criar gráficos de comparação
print("Criando gráficos de comparação de modelos...")

comparison_charts = create_comparison_charts(
    results_list=[pooled_results, fe_results, re_results],
    model_names=['Pooled OLS', 'Fixed Effects', 'Random Effects'],
    theme='professional',
    include_html=False
)

print(f"\n✓ {len(comparison_charts)} gráficos de comparação criados:")
for chart_name in comparison_charts.keys():
    print(f"  - {chart_name}")

Criando gráficos de comparação de modelos...

✓ 3 gráficos de comparação criados:
  - coefficients
  - fit_comparison
  - ic_comparison


In [37]:
from IPython.display import HTML, display

# Coefficient Comparison - Comparação dos coeficientes
print("Coefficient Comparison - Comparação dos Coeficientes")
print("="*70)
print("Compare as estimativas dos coeficientes entre modelos")
print()
display(HTML(comparison_charts['coefficients'].to_html()))

Coefficient Comparison - Comparação dos Coeficientes
Compare as estimativas dos coeficientes entre modelos



In [38]:
from IPython.display import HTML, display

# Forest Plot - Intervalos de confiança
print("Forest Plot - Intervalos de Confiança")
print("="*70)
print("Visualize a incerteza das estimativas")
print()
display(HTML(comparison_charts['coefficients'].to_html()))

Forest Plot - Intervalos de Confiança
Visualize a incerteza das estimativas



In [39]:
from IPython.display import HTML, display

# Model Fit Comparison - Comparação de ajuste
print("Model Fit Comparison - Comparação de Ajuste")
print("="*70)
print("Compare R², MSE, e outras métricas de ajuste")
print()
display(HTML(comparison_charts['fit_comparison'].to_html()))

Model Fit Comparison - Comparação de Ajuste
Compare R², MSE, e outras métricas de ajuste



In [40]:
from IPython.display import HTML, display

# Information Criteria - Critérios de informação
print("Information Criteria - Critérios de Informação")
print("="*70)
print("Menor AIC/BIC = melhor modelo")
print()
display(HTML(comparison_charts['ic_comparison'].to_html()))

Information Criteria - Critérios de Informação
Menor AIC/BIC = melhor modelo



### 4.2 Gerar Relatório HTML de Comparação

Criar relatório HTML comparando todos os modelos:

In [41]:
# Criar ReportManager
report_manager = ReportManager()

# Gerar relatório HTML de comparação
print("Gerando relatório HTML de comparação de modelos...")

# Preparar dados para comparação
comparison_data = {
    'models': {
        'Pooled OLS': {
            'params': pooled_results.params.to_dict(),
            'rsquared': pooled_results.rsquared,
            'nobs': pooled_results.nobs
        },
        'Fixed Effects': {
            'params': fe_results.params.to_dict(),
            'rsquared': fe_results.rsquared_within,
            'nobs': fe_results.nobs
        },
        'Random Effects': {
            'params': re_results.params.to_dict(),
            'rsquared': re_results.rsquared,
            'nobs': re_results.nobs
        }
    }
}

html_report = report_manager.generate_comparison_report(
    comparison_data=comparison_data,
    title='Comparação de Modelos - Dados Grunfeld'
)

# Salvar relatório
with open('output/reports/model_comparison.html', 'w', encoding='utf-8') as f:
    f.write(html_report)

print("\n✅ Relatório de comparação gerado!")
print("\nLocalização: output/reports/model_comparison.html")
print("\nO relatório inclui:")
print("  - Comparação visual de coeficientes")
print("  - Métricas de ajuste lado a lado")
print("  - Comparação detalhada")

Gerando relatório HTML de comparação de modelos...

✅ Relatório de comparação gerado!

Localização: output/reports/model_comparison.html

O relatório inclui:
  - Comparação visual de coeficientes
  - Métricas de ajuste lado a lado
  - Comparação detalhada


---

## 5. Temas e Personalização {#temas}

### Temas Disponíveis

O PanelBox oferece 3 temas profissionais:

| Tema | Uso Recomendado | Características |
|------|-----------------|----------------|
| **Professional** | Relatórios corporativos, apresentações executivas | Cores sóbrias, design limpo |
| **Academic** | Artigos científicos, dissertações | Alta legibilidade, preto e branco friendly |
| **Presentation** | Apresentações, slides | Cores vibrantes, alto contraste |

Vamos comparar os temas:

In [42]:
# Criar mesmos gráficos com diferentes temas
print("Criando gráficos com diferentes temas...\n")

# Tema Professional
print("1. Tema PROFESSIONAL")
charts_professional = create_validation_charts(
    validation_data=validation_results,
    theme='professional',
    include_html=False
)

# Tema Academic
print("2. Tema ACADEMIC")
charts_academic = create_validation_charts(
    validation_data=validation_results,
    theme='academic',
    include_html=False
)

# Tema Presentation
print("3. Tema PRESENTATION")
charts_presentation = create_validation_charts(
    validation_data=validation_results,
    theme='presentation',
    include_html=False
)

print("\n✓ Gráficos criados com 3 temas diferentes")

Criando gráficos com diferentes temas...

1. Tema PROFESSIONAL
2. Tema ACADEMIC
3. Tema PRESENTATION

✓ Gráficos criados com 3 temas diferentes


In [43]:
from IPython.display import HTML, display

# Comparar tema Professional
print("Tema PROFESSIONAL - Relatórios Corporativos")
print("="*70)
display(HTML(charts_professional['test_overview'].to_html()))

Tema PROFESSIONAL - Relatórios Corporativos


In [44]:
from IPython.display import HTML, display

# Comparar tema Academic
print("Tema ACADEMIC - Publicações Científicas")
print("="*70)
display(HTML(charts_academic['test_overview'].to_html()))

Tema ACADEMIC - Publicações Científicas


In [45]:
from IPython.display import HTML, display

# Comparar tema Presentation
print("Tema PRESENTATION - Apresentações e Slides")
print("="*70)
display(HTML(charts_presentation['test_overview'].to_html()))

Tema PRESENTATION - Apresentações e Slides


---

## 6. Exportação de Gráficos {#exportacao}

### Formatos Suportados

Todos os gráficos podem ser exportados para múltiplos formatos:

| Formato | Tipo | Uso Recomendado | Características |
|---------|------|-----------------|----------------|
| **PNG** | Raster | Web, apresentações | Boa qualidade, tamanho médio |
| **SVG** | Vector | Impressão, publicações | Escalável infinitamente, arquivos pequenos |
| **PDF** | Document | Artigos científicos | Alta qualidade, portável |
| **JPEG** | Raster | Web (fotos) | Comprimido, menor qualidade |
| **WEBP** | Raster | Web moderna | Melhor compressão |

### 6.1 Exportar Gráfico Individual

In [46]:
# Exportar um gráfico individual
print("Exportando gráfico individual em múltiplos formatos...\n")

chart = validation_charts['test_overview']

# PNG
export_chart(
    chart,
    'output/exports/validation_overview.png',
    width=1200,
    height=800
)
print("✓ PNG: output/exports/validation_overview.png")

# SVG (vetorial - ideal para publicações)
export_chart(
    chart,
    'output/exports/validation_overview.svg',
    width=1200,
    height=800
)
print("✓ SVG: output/exports/validation_overview.svg")

# PDF
export_chart(
    chart,
    'output/exports/validation_overview.pdf',
    width=1200,
    height=800
)
print("✓ PDF: output/exports/validation_overview.pdf")

# Alta resolução para impressão (scale=2.0)
export_chart(
    chart,
    'output/exports/validation_overview_hires.png',
    width=2400,
    height=1600,
    scale=2.0
)
print("✓ High-res PNG: output/exports/validation_overview_hires.png")

print("\n✅ Gráfico exportado em 4 formatos!")

Exportando gráfico individual em múltiplos formatos...

✓ PNG: output/exports/validation_overview.png
✓ SVG: output/exports/validation_overview.svg
✓ PDF: output/exports/validation_overview.pdf
✓ High-res PNG: output/exports/validation_overview_hires.png

✅ Gráfico exportado em 4 formatos!


### 6.2 Exportação em Lote

Exportar múltiplos gráficos de uma vez:

In [47]:
# Exportar todos os gráficos de validação como PNG
print("Exportação em lote - todos os gráficos de validação...\n")

paths = export_charts(
    charts=validation_charts,
    output_dir='output/exports/validation',
    format='png',
    width=1200,
    height=800,
    prefix='val_'
)

print("Arquivos exportados:")
for chart_name, file_path in paths.items():
    print(f"  ✓ {chart_name}: {file_path}")

print(f"\n✅ {len(paths)} gráficos exportados!")

Exportação em lote - todos os gráficos de validação...

Arquivos exportados:
  ✓ test_overview: output/exports/validation/val_test_overview.png
  ✓ pvalue_distribution: output/exports/validation/val_pvalue_distribution.png
  ✓ test_statistics: output/exports/validation/val_test_statistics.png

✅ 3 gráficos exportados!


### 6.3 Exportação Multi-Formato

Exportar mesmos gráficos em múltiplos formatos simultaneamente:

In [48]:
# Exportar em PNG, SVG e PDF simultaneamente
print("Exportação multi-formato...\n")

all_paths = export_charts_multiple_formats(
    charts=validation_charts,
    output_dir='output/exports/multi_format',
    formats=['png', 'svg', 'pdf'],
    width=1200,
    height=800
)

# Mostrar arquivos criados
for format_name, paths in all_paths.items():
    print(f"\n{format_name.upper()} exports:")
    for chart_name, file_path in paths.items():
        print(f"  ✓ {file_path}")

total_files = sum(len(paths) for paths in all_paths.values())
print(f"\n✅ {total_files} arquivos exportados em {len(all_paths)} formatos!")

Exportação multi-formato...


PNG exports:
  ✓ output/exports/multi_format/test_overview.png
  ✓ output/exports/multi_format/pvalue_distribution.png
  ✓ output/exports/multi_format/test_statistics.png

SVG exports:
  ✓ output/exports/multi_format/test_overview.svg
  ✓ output/exports/multi_format/pvalue_distribution.svg
  ✓ output/exports/multi_format/test_statistics.svg

PDF exports:
  ✓ output/exports/multi_format/test_overview.pdf
  ✓ output/exports/multi_format/pvalue_distribution.pdf
  ✓ output/exports/multi_format/test_statistics.pdf

✅ 9 arquivos exportados em 3 formatos!


### 6.4 Exportação para Publicação

Configurações otimizadas para diferentes usos:

In [49]:
# Configurações para diferentes casos de uso
print("Exportando para diferentes casos de uso...\n")

chart = validation_charts['test_overview']

# 1. Para apresentação (16:9)
export_chart(
    chart,
    'output/exports/presentation_16x9.png',
    width=1920,
    height=1080
)
print("✓ Apresentação (16:9): 1920x1080")

# 2. Para publicação científica (alta resolução)
export_chart(
    chart,
    'output/exports/publication.pdf',
    width=2400,
    height=1600
)
print("✓ Publicação (PDF): 2400x1600")

# 3. Para web (tamanho otimizado)
export_chart(
    chart,
    'output/exports/web.webp',
    width=1200,
    height=800
)
print("✓ Web (WEBP): 1200x800")

# 4. Para impressão (SVG vetorial)
export_chart(
    chart,
    'output/exports/print.svg',
    width=1200,
    height=800
)
print("✓ Impressão (SVG vetorial): infinitamente escalável")

print("\n✅ Todos os formatos exportados!")
print("\n💡 Dica: Use SVG/PDF para impressão, PNG para web, WEBP para web moderna")

Exportando para diferentes casos de uso...

✓ Apresentação (16:9): 1920x1080
✓ Publicação (PDF): 2400x1600
✓ Web (WEBP): 1200x800
✓ Impressão (SVG vetorial): infinitamente escalável

✅ Todos os formatos exportados!

💡 Dica: Use SVG/PDF para impressão, PNG para web, WEBP para web moderna


---

## 7. Workflows Automatizados {#automatizacao}

### 7.1 Análise Completa Automatizada

Criar um workflow completo que gera todos os relatórios automaticamente:

In [50]:
# Executar validação completa
from panelbox.validation import ValidationSuite

print("Executando validação completa...")

# Criar ValidationSuite a partir dos resultados
validator = ValidationSuite(fe_results)

# Executar todos os testes
validation_report = validator.run(tests='all', verbose=True)

# Converter ValidationReport para formato de visualização
# O ValidationReport tem um método to_dict() que retorna os resultados
report_dict = validation_report.to_dict()

# Criar estrutura de dados para visualização
validation_results = {
    'tests': [],
    'summary': {
        'total_tests': 0,
        'tests_passed': 0,
        'tests_failed': 0,
        'pass_rate': 0.0
    },
    'categories': {}
}

# Processar cada categoria de testes
for category in ['specification', 'serial_correlation', 'heteroskedasticity', 'cross_sectional']:
    if category in report_dict:
        validation_results['categories'][category] = []
        for test_name, test_result in report_dict[category].items():
            if isinstance(test_result, dict):
                test_info = {
                    'name': test_name,
                    'statistic': test_result.get('statistic', 0.0),
                    'pvalue': test_result.get('pvalue', 1.0),
                    'result': 'Pass' if test_result.get('pvalue', 1.0) > 0.05 else 'Fail',
                    'category': category
                }
                validation_results['tests'].append(test_info)
                validation_results['categories'][category].append(test_info)

# Calcular resumo
validation_results['summary']['total_tests'] = len(validation_results['tests'])
validation_results['summary']['tests_passed'] = sum(
    1 for t in validation_results['tests'] if t['result'] == 'Pass'
)
validation_results['summary']['tests_failed'] = (
    validation_results['summary']['total_tests'] - 
    validation_results['summary']['tests_passed']
)
if validation_results['summary']['total_tests'] > 0:
    validation_results['summary']['pass_rate'] = (
        100.0 * validation_results['summary']['tests_passed'] / 
        validation_results['summary']['total_tests']
    )

print(f"\n✓ {len(validation_results['tests'])} testes executados")
print(f"\nResumo:")
print(f"  - Total de testes: {validation_results['summary']['total_tests']}")
print(f"  - Testes aprovados: {validation_results['summary']['tests_passed']}")
print(f"  - Testes falhados: {validation_results['summary']['tests_failed']}")
print(f"  - Taxa de aprovação: {validation_results['summary']['pass_rate']:.1f}%")

Executando validação completa...
Running RESET test...
Running Wooldridge AR test...
Running Breusch-Godfrey test...
Running Baltagi-Wu LBI test...
Running Modified Wald test...
Running Breusch-Pagan test...
Running White test...
Running Pesaran CD test...
Running Breusch-Pagan LM test...
Running Frees test...

✓ 0 testes executados

Resumo:
  - Total de testes: 0
  - Testes aprovados: 0
  - Testes falhados: 0
  - Taxa de aprovação: 0.0%


### 7.2 Pipeline Batch para Múltiplos Modelos

Processar múltiplas especificações de modelo em lote:

In [51]:
# Criar ReportManager
report_manager = ReportManager()

# Gerar relatório HTML de comparação
print("Gerando relatório HTML de comparação de modelos...")

# Preparar dados para comparação
comparison_data = {
    'models': {
        'Pooled OLS': {
            'params': pooled_results.params.to_dict(),
            'rsquared': pooled_results.rsquared,
            'nobs': pooled_results.nobs
        },
        'Fixed Effects': {
            'params': fe_results.params.to_dict(),
            'rsquared': fe_results.rsquared_within,
            'nobs': fe_results.nobs
        },
        'Random Effects': {
            'params': re_results.params.to_dict(),
            'rsquared': re_results.rsquared,
            'nobs': re_results.nobs
        }
    }
}

html_report = report_manager.generate_comparison_report(
    comparison_data=comparison_data,
    title='Comparação de Modelos - Dados Grunfeld'
)

# Salvar relatório
with open('output/reports/model_comparison.html', 'w', encoding='utf-8') as f:
    f.write(html_report)

print("\n✅ Relatório de comparação gerado!")
print("\nLocalização: output/reports/model_comparison.html")
print("\nO relatório inclui:")
print("  - Comparação visual de coeficientes")
print("  - Métricas de ajuste lado a lado")
print("  - Comparação detalhada")

Gerando relatório HTML de comparação de modelos...

✅ Relatório de comparação gerado!

Localização: output/reports/model_comparison.html

O relatório inclui:
  - Comparação visual de coeficientes
  - Métricas de ajuste lado a lado
  - Comparação detalhada


---

## Resumo

### O que você aprendeu

✅ **Três tipos de relatórios**:
- Relatórios de Validação
- Diagnósticos Residuais
- Comparação de Modelos

✅ **Geração de relatórios**:
- Gráficos interativos individuais
- Relatórios HTML completos
- Exportação em múltiplos formatos

✅ **Temas profissionais**:
- Professional (corporativo)
- Academic (científico)
- Presentation (apresentações)

✅ **Exportação flexível**:
- PNG, SVG, PDF, JPEG, WEBP
- Exportação individual e em lote
- Multi-formato simultâneo

✅ **Automação**:
- Workflows completos
- Comparação em lote
- Pipeline automatizado

### Principais Comandos

```python
# Criar gráficos
charts = create_validation_charts(validation_data, theme='professional')
charts = create_residual_diagnostics(results, theme='professional')
charts = create_comparison_charts(results_list, model_names)

# Gerar relatórios HTML
report_mgr = ReportManager()
report_mgr.generate_validation_report(validation_data, output_file='report.html')
report_mgr.generate_residual_report(results, output_file='diagnostics.html')
report_mgr.generate_comparison_report(results_list, model_names, output_file='comparison.html')

# Exportar gráficos
export_chart(chart, 'output.png', width=1200, height=800)
export_charts(charts, 'output/', format='png')
export_charts_multiple_formats(charts, 'output/', formats=['png', 'svg', 'pdf'])
```

### Próximos Passos

- **Explore outros notebooks**:
  - [03_validation_complete.ipynb](./03_validation_complete.ipynb) - Testes de validação
  - [04_robust_inference.ipynb](./04_robust_inference.ipynb) - Inferência robusta
  - [07_real_world_case_study.ipynb](./07_real_world_case_study.ipynb) - Caso real

- **Personalize** os temas e gráficos para suas necessidades
- **Automatize** seus workflows de análise
- **Compartilhe** relatórios HTML interativos com colegas

---

## Recursos Adicionais

- 📚 [Documentação Completa](https://panelbox.readthedocs.io)
- 📊 [Guia de Exportação](../../desenvolvimento/REPORT/EXPORT_FUNCTIONALITY_GUIDE.md)
- 🎨 [Guia de Temas](../../desenvolvimento/REPORT/THEME_GUIDE.md)
- 💻 [Exemplo de Script](../export_charts_example.py)

---

*Crie relatórios profissionais com PanelBox v0.5.0!* 🚀
